<a href="https://colab.research.google.com/github/mad-scientist-in-training/BIOF509-Final-Project/blob/Project-Code/BIOF509_Final_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is a project to determine if speech patterns in Parkinson's patients vs healthy controls can be used to train a Support Vector Machine model to predict which patients belong to which category.

In [ ]:
#Code outline

In [ ]:
#to download the kaggle dataset, create a kaggle folder under google drive, create a new API token under account in kaggle, put the kaggle.json file into the kaggle folder in your drive, and mount the drive
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLEC_CONFIG_DIR'] = "content/gdrive/My Drive/kaggle_dataset"
%cd /content/gdrive/My Drive/kaggle_dataset
#copy the API command from kaggle and unzip the file
!kaggle datasets download -d ruslankl/early-biomarkers-of-parkinsons-disease
!unzip early-biomarkers-of-parkinsons-disease

In [ ]:
#manipulate/visualize dataset to keep features you are interested in and get rid of ones you don't care about
import pandas as pd
PD_df = pd.read_csv(r'dataset.csv')
print(PD_df)
len(PD_df)
PD_df.head()

In [ ]:
#drop the first 40 columns, as they contain info we don't need
PD_df_1 = PD_df.drop(PD_df.columns[1:41], axis=1)
print(PD_df_1)
#drop the participant codes containing RBD
PD_df_2 = PD_df_1.drop(index= [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79])
print(PD_df_2)
PD_df_2.columns


In [ ]:
#Participant code needs to be made into distinct classes: P (Parkinson's disease) vs H (Healthy Control), which will be the target
import numpy as np
PD_DF = PD_df_2.rename({' Participant  code ':'Participant_Type'}, axis=1)
print(PD_DF.columns)
PD_DF['class'] = PD_DF.Participant_Type.str[0]
print(PD_DF)
target = PD_DF.pop('class')
PD_DF.insert(0, 'class', target)
PD_DF_new = PD_DF.drop(PD_DF.columns[1], axis=1)
PD_DF_new

In [ ]:
#scale the data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
x = PD_DF_new.loc[:, PD_DF_new.columns != 'class'].values
x = StandardScaler().fit_transform(x)
print(x.shape)
print(np.mean(x))
print(np.std(x))
df = pd.DataFrame(x)
print(df)


In [ ]:
#run a PCA with two components, the same number as the classes of patients (healthy vs PD)
from sklearn.decomposition import PCA
pca_PD = PCA(n_components=2)
principleComponents_PD = pca_PD.fit_transform(x)

In [ ]:
#visualize the components
principle_PD_DF = pd.DataFrame(data = principleComponents_PD, columns= ['principle_component1', 'principle_component2'])
principle_PD_DF.tail()

,principle_component1,principle_component2
75,2.465902,1.455955
76,0.542998,0.834763
77,2.496417,-0.186708
78,0.036594,0.885698
79,2.104952,1.110782


In [ ]:
print('Explained variation between principle components: {}'.format(pca_PD.explained_variance_ratio_))

In [ ]:
#plot the principle components vs the classes - healthy vs PD
import matplotlib.pyplot as plt
import plotly.express as px
px.scatter(x=principle_PD_DF['principle_component1'], y=principle_PD_DF['principle_component2'], color= PD_DF_new['class'])



In [ ]:
#split the data into training and test sets
import sklearn
from sklearn.model_selection import train_test_split
X, y = 

In [ ]:
#create the knn object
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

In [ ]:
#fit the model to the training data

In [ ]:
#carry out model on testing data

In [ ]:
#calculate score for how well the model fits